In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/QAI4EO/')

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, Flatten, MaxPool2D, BatchNormalization, AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD

# Lenet-5

In [ ]:
kernel_size = 3
stride = 1

lenet_5_model = Sequential()
lenet_5_model.add(Conv2D(16, kernel_size=kernel_size, strides=stride, activation='relu', padding='same', input_shape = (64,64,3)))
lenet_5_model.add(AveragePooling2D())
lenet_5_model.add(Conv2D(32, kernel_size=kernel_size, strides=stride, activation='relu', padding='valid'))
lenet_5_model.add(AveragePooling2D())
lenet_5_model.add(Conv2D(64, kernel_size=kernel_size, strides=stride, activation='relu', padding='valid'))
lenet_5_model.add(AveragePooling2D())

lenet_5_model.add(Flatten())
lenet_5_model.add(Dense(8, activation='relu'))
lenet_5_model.add(Dense(16, activation='relu'))
lenet_5_model.add(Dense(10, activation='softmax'))

lenet_5_model.compile(optimizer=SGD(lr=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])
lenet_5_model.summary()

# Load Dataset

In [ ]:
from DatasetHandler import DatasetHandler

dataset_root = '/content/drive/MyDrive/QAI4EO/datasets/EuroSAT/EuroSAT'
handler = DatasetHandler(dataset_root)

In [ ]:
classes = []
for i, c in enumerate(handler.classes):
  cl = c.split('/')[-1]
  classes.append(cl)
  print('Class %d -> %s' % (i, cl))

In [ ]:
imgs_path, imgs_label = handler.load_paths_labels(dataset_root, classes=classes)
print('Dataset images:', len(imgs_path), 'Dataset labels:', len(imgs_label))
print('Dataset sample ->', imgs_path[0], imgs_label[0])

In [ ]:
train_imgs, train_labels, val_images, val_labels = handler.train_validation_split(imgs_path, imgs_label, split_factor=0.2)
print('X_train shape:', train_imgs.shape, 'Y_train shape:', train_labels.shape)
print('  X_val shape: ', val_images.shape, '  Y_val shape: ', val_labels.shape)

In [ ]:
val_gen = handler.cnn_data_loader(val_images, val_labels, batch_size=batch_size, img_shape=(64,64,3), n_classes=10)

In [ ]:
batch_size = 1
epochs = 100

train_gen = handler.cnn_data_loader(train_imgs, train_labels, batch_size=batch_size, img_shape=(64,64,3), n_classes=10)
val_gen = handler.cnn_data_loader(val_images, val_labels, batch_size=batch_size, img_shape=(64,64,3), n_classes=10)


es = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

history = lenet_5_model.fit(train_gen,
                            steps_per_epoch = len(train_labels)//batch_size,
                            epochs = epochs,
                            validation_data = val_gen,
                            validation_steps = len(val_labels)//batch_size,
                            callbacks = [es])

In [ ]:
lenet_5_model.save('/content/drive/MyDrive/QAI4EO/lenet5.h5')

In [ ]:
import numpy as np
val_gen = iter(handler.cnn_data_loader(val_images, val_labels, batch_size=batch_size, img_shape=(64,64,3), n_classes=10))


predictions = []
targets = []

for ct in range(len(val_images)):
    data, target = next(val_gen)
    output = lenet_5_model.predict(data)
    
    predictions.append(np.argmax(output))
    targets.append(np.argmax(target))
  

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
cm = confusion_matrix(predictions, targets, normalize='true')

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (12,10))

cmd = ConfusionMatrixDisplay(cm, display_labels=classes)
cmd.plot(ax=axes, cmap='Blues', xticks_rotation='vertical')
print('S2')
print('Accuracy:', cm.diagonal(), 'mean: ', cm.diagonal().mean())
print(classification_report(targets, predictions, target_names=classes, digits=4))
axes.get_images()[0].set_clim(0, 1)
plt.show()
plt.close()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import numpy as np

In [ ]:
cm = np.array([[76.0, 13.0],[11.0, 60.0]])
cm2 = np.array([[0.85, 0.15],[0.15, 0.85]])

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 30})

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30,10))

cmd = ConfusionMatrixDisplay(cm, display_labels=['No Eruption', 'Eruption'])
cmd.plot(ax=axes[0], cmap='Blues', xticks_rotation='horizontal')
#axes[0].get_images()[0].set_clim(0, 1)
axes[0].set_title('Confusion Matrix')

cmd = ConfusionMatrixDisplay(cm2, display_labels=['No Eruption', 'Eruption'])
cmd.plot(ax=axes[1], cmap='Blues', xticks_rotation='horizontal')
axes[1].get_images()[0].set_clim(0, 1)
axes[1].set_title('Normalized Confusion Matrix')
plt.show()
plt.close()

In [ ]:
cm = np.array([[146, 67],[22, 51]]).astype(np.uint16)
cm2 = np.array([[0.690, 0.31],[0.30, 0.70]])

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 30})

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30,10))
cmd = ConfusionMatrixDisplay(cm, display_labels=['No Landslide', 'Landslide'])
cmd.plot(ax=axes[0], cmap='Blues', xticks_rotation='horizontal', values_format = '')
#axes[0].get_images()[0].set_clim(0, 1)
axes[0].set_title('Confusion Matrix')


cmd = ConfusionMatrixDisplay(cm2, display_labels=['No Landslide', 'Landslide'])
cmd.plot(ax=axes[1], cmap='Blues', xticks_rotation='horizontal')
axes[1].get_images()[0].set_clim(0, 1)
axes[1].set_title('Normalized Confusion Matrix')
plt.show()
plt.close()


In [ ]:
cmd.plot?